In [1]:
import requests
from bs4 import BeautifulSoup

raw = requests.get("https://search.naver.com/search.naver?where=news&query=직방")
html = BeautifulSoup(raw.text, "html.parser")

In [2]:
raw

<Response [200]>

In [6]:
import requests
import json
import time   
import pandas as pd

url = 'https://openapi.naver.com/v1/search/news?'
clientId = "*****"
clientSecret = "*****"
header = {
    "X-Naver-Client-Id":clientId,
    "X-Naver-Client-secret":clientSecret,
}

#start가 100에서 처음 등장하는 것이 start 1일때 newslist[99]임
#들어가야할 start 값 -> 1, 101, 201, 301, 401, 
start = 1
while start <= 1000 : #start는 1000개가 최대
    time.sleep(0.1)
    params = {'query':"메타버스" , 'display':100, 'start':start, 'sort':'date'}
    r = requests.get(url, params = params, headers = header)
    time.sleep(0.1)
    j = r.json() 
    newslist = j['items'] 
    #df = df.append(newslist)
    start += 100
    time.sleep(0.1)
    print(j)


{'lastBuildDate': 'Mon, 19 Sep 2022 22:38:00 +0900', 'total': 268338, 'start': 1, 'display': 100, 'items': [{'title': '경북도, &apos;<b>메타버스</b> 통해 제15회 치매극복의 날 기념&apos; 개최', 'originallink': 'https://www.dnews.co.kr/uhtml/view.jsp?idxno=202209191546267030927', 'link': 'https://www.dnews.co.kr/uhtml/view.jsp?idxno=202209191546267030927', 'description': '경북도가 19일 <b>메타버스</b>(ZEP) 공간 속에서 25개 치매안심센터 관계자 등 300여명이 참석한 가운데 &apos;따뜻한 동행... 이번 기념식은 기존에 강당이나 회의장 등에 모여 개최됐던 기념식 대신 사무실이나 자택에서 <b>메타버스</b>에... ', 'pubDate': 'Mon, 19 Sep 2022 22:32:00 +0900'}, {'title': 'SK텔레콤 ‘이프랜드’서 초실감 <b>메타버스</b> 콘서트[포토뉴스]', 'originallink': 'https://www.khan.co.kr/economy/economy-general/article/202209192202035', 'link': 'https://n.news.naver.com/mnews/article/032/0003174030?sid=101', 'description': 'SK텔레콤은 <b>메타버스</b> 플랫폼 이프랜드에서 초실감 가상 콘서트 &lt;메타홍대 뮤직투어&gt;를 오는 25일까지 진행한다고 19일 밝혔다. 100대가 넘는 카메라로 360도를 동시에 촬영하는 영상 제작 기법인 볼류메트릭 기술이... ', 'pubDate': 'Mon, 19 Sep 2022 22:11:00 +0900'}, {'title': '포스코건설, 가상사옥 메타스페이스 오픈'

In [5]:
r.json()

{'errorMessage': 'Scope Status Invalid : Authentication failed. (인증에 실패했습니다.)',
 'errorCode': '024'}